<font color = yellow> mid - train + 추가내용 <<0523 수정>> </font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train=pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/Augmentation_data.csv')

In [ ]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업0103
1,광업(05~08),B,광업0508
2,제조업(10~34),C,제조업1034
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업35
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업3639


In [ ]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업
1,광업(05~08),B,광업
2,제조업(10~34),C,제조업
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업


In [ ]:
plus["mid"] = plus["Target"].str[1:3]
plus

,Dis,Target,BZ_PPOS_ITM_CTT,mid
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업,
1,광업(05~08),B,광업,
2,제조업(10~34),C,제조업,
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업,
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업,
...,...,...,...,...
2016,가구 내 고용활동,T97000,가구 내 고용활동,97
2017,자가 소비를 위한 가사 생산 활동,T98100,자가 소비를 위한 가사 생산 활동,98
2018,자가 소비를 위한 가사 서비스 활동,T98200,자가 소비를 위한 가사 서비스 활동,98
2019,주한 외국 공관,U99001,주한 외국 공관,99


In [ ]:
# 추가내용 파일 - 해당 분류에 맞게 삭제 
for i in range(len(plus)):
  if len(plus['mid'][i])<2:
    plus=plus.drop(index=i)

print(len(plus))

1999


In [ ]:
plus['mid'].unique()

array(['01', '02', '03', '05', '06', '07', '08', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '36',
       '37', '38', '39', '41', '42', '45', '46', '47', '49', '50', '51',
       '52', '55', '56', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '68', '70', '71', '72', '73', '74', '75', '76', '84', '85',
       '86', '87', '90', '91', '94', '95', '96', '97', '98', '99', '35'],
      dtype=object)

In [ ]:
data_mid = plus.loc[:,['BZ_PPOS_ITM_CTT', 'mid']]
data_mid.tail()

,BZ_PPOS_ITM_CTT,mid
2016,가구 내 고용활동,97
2017,자가 소비를 위한 가사 생산 활동,98
2018,자가 소비를 위한 가사 서비스 활동,98
2019,주한 외국 공관,99
2020,기타 국제 및 외국기관,99


In [ ]:
for i in range(len(train)):
  if len(train['KSIC10_BZC_CD'][i])<3:
    train=train.drop(index=i)

print(len(train))

In [ ]:
train['mid']=train['KSIC10_BZC_CD'].str[1:3]
train.head()

In [ ]:
trn = pd.concat([train, data_mid], ignore_index = True)
trn.tail()

In [ ]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'mid']]
trn.tail()

In [ ]:
trn =trn.dropna(axis=0)
len(trn)

In [ ]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.tail()

In [ ]:
trn_shuffle.to_csv('/content/drive/MyDrive/애쓰는 감자/data/mid_data/train_data_mid.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['mid'])
trn_shuffle['mid_idx'] = encoder.transform(trn_shuffle['mid'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'mid_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('/content/drive/MyDrive/애쓰는 감자/data/mid_data/train_mid.txt', sep='\t', index=True)